<a href="https://colab.research.google.com/github/shirsneh/virtual_clothes_try_on_assistant/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the enviorment

In [1]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY {CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
!mkdir openpose/images openpose/videos

sed: -e expression #1, char 102: unterminated `s' command
Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../00-liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selecting previously unselected package libgflags2.2.
Preparing to unpack .../01-libgflags2.2_2.2.2-2_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-2) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../02-libgflags-dev_2.2.2-2_amd64.deb ...
Unpacking libgflags-dev (2.2.2-2) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../03-libgoogle-glog0v5_0.5.0+really0.4.0-2_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.5.0+really0.4.0-2) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../04-libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Selecting previou

In [2]:
!pip install --upgrade --no-cache-dir gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [3]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2P0zpWK6ARB2COFdET2OJm1sFFb_6kyEQrMiGAWyLDmLWzWwf

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=c884fb0eb0c0eb4a764f8b891aa05cdb00419791ef9a10c946381814839540a4
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
%cd /content/
!rm -rf virtual_clothes_try_on_assistant
!git clone https://github.com/shirsneh/virtual_clothes_try_on_assistant.git
os.makedirs("/content/virtual_clothes_try_on_assistant/checkpoints")

/content
Cloning into 'virtual_clothes_try_on_assistant'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 337 (delta 8), reused 11 (delta 5), pack-reused 321
Receiving objects: 100% (337/337), 4.37 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [5]:
!gdown --id 18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6 --output /content/virtual_clothes_try_on_assistant/checkpoints/alias_final.pth
!gdown --id 1uDRPY8gh9sHb3UDonq6ZrINqDOd7pmTz --output /content/virtual_clothes_try_on_assistant/checkpoints/gmm_final.pth
!gdown --id 1d7lZNLh51Qt5Mi1lXqyi6Asb2ncLrEdC --output /content/virtual_clothes_try_on_assistant/checkpoints/seg_final.pth


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6
From (redirected): https://drive.google.com/uc?id=18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6&confirm=t&uuid=06e2c770-5904-482a-9eba-7594045f545f
To: /content/virtual_clothes_try_on_assistant/checkpoints/alias_final.pth
100% 402M/402M [00:02<00:00, 157MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1uDRPY8gh9sHb3UDonq6ZrINqDOd7pmTz
To: /content/virtual_clothes_try_on_assistant/checkpoints/gmm_final.pth
100% 76.2M/76.2M [00:00<00:00, 202MB/s]
/usr/local/lib/python3.10

In [6]:
!gdown --id 1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx --output /content/virtual_clothes_try_on_assistant/cloth_segm_u2net_latest.pth --no-cookies

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx
From (redirected): https://drive.google.com/uc?id=1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx&confirm=t&uuid=1a765a41-b281-457f-8034-fc4adddffa6d
To: /content/virtual_clothes_try_on_assistant/cloth_segm_u2net_latest.pth
100% 177M/177M [00:00<00:00, 186MB/s]


In [7]:
%cd /content/
!pip install ninja

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 3.4 MB/s eta 0:00:00


In [8]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints

Cloning into 'Self-Correction-Human-Parsing'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (719/719), done.
remote: Compressing objects: 100% (568/568), done.
remote: Total 719 (delta 149), reused 611 (delta 140), pack-reused 0
Receiving objects: 100% (719/719), 3.88 MiB | 10.03 MiB/s, done.
Resolving deltas: 100% (149/149), done.
/content/Self-Correction-Human-Parsing


In [9]:
# downloading LIP dataset model
!gdown --id 1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
!mv /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth /content/Self-Correction-Human-Parsing/checkpoints/final.pth

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=26267ba4-fcfe-4156-97c7-8f26c11855d4
To: /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth
100% 267M/267M [00:02<00:00, 114MB/s]


In [10]:
%cd /content/


/content


In [11]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2023-08-30 20:08:04--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.245.55, 104.16.246.55, 2606:4700::6810:f537, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.245.55|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2023-08-30 20:08:05--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 90.4M 1s
    50K .......... .......... .......... .......... ..........  0%

In [12]:
%%bash
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    urllib3-1.26.16            |   py38h06a4308_0         182 KB
    pysocks-1.7.1              |   py38h06a4308_0          31 KB
    ld_impl_linux-64-2.38      |       h1181459_1         732 KB
    tk-8.6.12                  |       h1ccaba5_0         3.3 MB
    conda-package-handling-2.2.0|   py38h06a4308_0         270 KB
    cffi-1.15.0                |   py38hd667e15_1         226 KB
    pip-23.2.1                 |   py38h06a4308_0         2.5 MB
    ca-certificates-2023.05.30 |       h06a4308_0         127 KB
    tqdm-4.65.0                |   py38hb070fc8_0         121 KB
    setuptools-68.0.0          |   py38h06a4308_0         1.0 MB
    zstandard-0.15.2           |   py38h27cfd23_0  

urllib3-1.26.16      |  182 KB | ########## | 100% 
pysocks-1.7.1        |   31 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
conda-package-handli |  270 KB | ########## | 100% 
cffi-1.15.0          |  226 KB | ########## | 100% 
pip-23.2.1           |  2.5 MB | ########## | 100% 
ca-certificates-2023 |  127 KB | ########## | 100% 
tqdm-4.65.0          |  121 KB | ########## | 100% 
setuptools-68.0.0    |  1.0 MB | ########## | 100% 
zstandard-0.15.2     |  599 KB | ########## | 100% 
cryptography-41.0.2  |  2.0 MB | ########## | 100% 
conda-package-stream |   26 KB | ########## | 100% 
jsonpatch-1.32       |   14 KB | ########## | 100% 
toolz-0.12.0         |   99 KB | ########## | 100% 
boltons-23.0.0       |  421 KB | ########## | 100% 
packaging-23.0       |   64 KB | ########## | 100% 
pycosat-0.6.3        |  107 KB | ########## | 100% 
pycparser-2.21       |   94 KB | ########## | 100% 
brotlipy-0.7

In [13]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

In [14]:
!conda install --channel conda-forge featuretools --yes

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [15]:
!pip install --upgrade mediapipe


  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/dc/d8/4ea533f021eae54f6449a49e8456d2d045708e6d1081313bad7a1ead3386/mediapipe-0.10.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.0 MB/s eta 0:00:00
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/b4/c2/f74e0deb26379aead0956a6ecf9acd4587debba0c7abe4bd8fe53fe04ec2/matplotlib-3.7.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/4b/5e/b3d986289c4bbb57fbad

In [16]:
!pip install opencv-python torchgeometry

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/f5/d0/2e455d894ec0d6527e662ad55e70c04f421ad83a6fd0a54c3dd73c411282/opencv_python-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/52/90/45223db4e1df30ff14e8aebf9a1bf0222da2e7b49e53692c968f36817812/filelock-3.12.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.9

In [17]:
!pip install torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 25.6 MB/s eta 0:00:00


In [18]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

In [19]:
!pip install --upgrade numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/98/5d/5738903efe0ecb73e51eb44feafba32bdba2081263d40c5043568ff60faf/numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


In [20]:
def make_dir():
  os.system("cd /content/ && mkdir inputs && cd inputs && mkdir test && cd test && mkdir cloth cloth-mask image image-parse mediapipe_img mediapipe_json")

# **Running the App**

In [21]:
from flask import Flask, request, send_file, jsonify
from flask_ngrok import run_with_ngrok
from PIL import Image
import base64
import io
from pdb import set_trace;
import os
from PIL import Image
import numpy as np
from collections import OrderedDict
import subprocess



app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return jsonify("Your API has deployed successfully");


@app.route("/api/transform", methods=['POST'])
def begin():
    make_dir()

    cloth = request.files['cloth']
    model = request.files['model']
    print("Data received")

    cloth_path = "/content/inputs/test/cloth/cloth.jpg"
    model_path = "/content/inputs/test/image/model.jpg"

    cloth = Image.open(cloth.stream)
    model = Image.open(model.stream)

    cloth.save(cloth_path)
    model.save(model_path)

    print("Images saved")

    # Run the transformation process synchronously
    result = subprocess.run(["python", "/content/virtual_clothes_try_on_assistant/run.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print("Subprocess stdout:")
    print(result.stdout)

    print("Subprocess stderr:")
    print(result.stderr)

    # Loading output
    if not os.path.exists("/content/output") or not os.listdir("/content/output"):
        op_path = "/content/virtual_clothes_try_on_assistant/client/static/images/error.jpg"
    else:
        op_filename = os.listdir("/content/output")[0]
        op_path = os.path.join("/content/output", op_filename)

    with open(op_path, 'rb') as op_file:
        op_data = op_file.read()

    os.system("rm -rf /content/output/")

    return send_file(io.BytesIO(op_data), mimetype='image/gif')



In [22]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://dcef-34-90-218-102.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
Data received
Images saved


INFO:werkzeug:127.0.0.1 - - [30/Aug/2023 20:17:52] "POST /api/transform HTTP/1.1" 200 -


Subprocess stdout:
checkpoints loaded from path: cloth_segm_u2net_latest.pth
Evaluating total class number 20 with ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat', 'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm', 'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
Namespace(alias_checkpoint='alias_final.pth', batch_size=1, checkpoint_dir='/content/virtual_clothes_try_on_assistant/checkpoints', dataset_dir='/content/inputs', dataset_list='test_pairs.txt', dataset_mode='test', display_freq=1, gmm_checkpoint='gmm_final.pth', grid_size=5, init_type='xavier', init_variance=0.02, load_height=1024, load_width=768, name='output', ngf=64, norm_G='spectralaliasinstance', num_upsampling_layers='most', save_dir='/content/', seg_checkpoint='seg_final.pth', semantic_nc=13, shuffle=False, workers=1)
Network [SegGenerator] was created. Total number of parameters: 34.5 million. To see the architecture, print(network).
Network [ALIAS